In [1]:
from pyspark import SparkContext, SparkConf
from fnvhash import fnv1a_32

conf = SparkConf().setMaster("local[*]")
sc = SparkContext(conf=conf)

In [58]:
class Shingling():
    shinglings = []
    rdd = []
    k = 2
    
    def __init__(self, document, k):
        self.rdd = sc.textFile(document)
        self.k = k
        self.shinglings = self.rdd.map(Shingling.createShinglings(self)).flatMap(lambda list: list).collect()

    @staticmethod
    def createShinglings(Shingling):
        k = Shingling.k
        def _createShinglings(document):
            shingles = []
            for i in range(0,len(document)-k+1):
                currentShingle = document[i:i+k]
                #Convert shingles to integer representation
                hashed_shingle = fnv1a_32(str.encode(currentShingle))
                #Append hashed value to list
                shingles.append(hashed_shingle)
            return shingles
        return _createShinglings

    def getShinglings(self):
        return set(self.shinglings)

document1 = Shingling('titanic.csv',3)
document1.getShinglings()

{3356164097,
 2634973196,
 3019276302,
 1319632911,
 1449164816,
 4120969238,
 2786983959,
 1989935128,
 3327754271,
 1180336161,
 820805666,
 3837460514,
 3820945445,
 3979214893,
 2540798003,
 1242136629,
 3048833087,
 3437264960,
 2623242308,
 2738225220,
 2384986197,
 2344157269,
 2931851353,
 973308001,
 1692696674,
 1152254079,
 612434064,
 3717333137,
 2507210905,
 2552332442,
 2378006697,
 2765979818,
 3902701742,
 3086975158,
 1200783562,
 312737995,
 569966797,
 2507866319,
 4032561364,
 3436282069,
 3001057493,
 1089765596,
 2491351263,
 2185199840,
 3208577252,
 442597617,
 4018503923,
 2055831797,
 2715713782,
 2201878778,
 1445495034,
 2847080701,
 1266876673,
 1939013891,
 2432205065,
 2716041492,
 2126512414,
 346587423,
 1351319841,
 809042211,
 839778597,
 338329895,
 3394437416,
 3858891060,
 2028208447,
 2365096256,
 476447045,
 3787489608,
 1132364104,
 1302561102,
 3164766543,
 799768938,
 842891639,
 4252270983,
 1067254158,
 2405073294,
 2641658266,
 1336410530,

In [ ]:
def Shingling(document, k):
    shingles = []
    shingles_hashed = []
    for i in range(0,len(document)):
        if i != len(document)-1:
            x = document[i:i+k]
            x = x.lower()
            if x not in shingles:
                shingles.append(x)
                shingles_hashed.append(hash(x))
        else:
            continue
    return shingles#,shingles_hashed

#shingles, shingles_hashed = Shingling(, 2)

def testfunc(s):
    return Shingling(s, 3)

rdd = sc.textFile("titanic.csv")
x = rdd.map(testfunc).flatMap(lambda list: list).collect()
x

#def CompareSets(setA,setB):

#    return

#def MinHashing():
    
#    return

#def CompareSignatures():
    
#    return

In [47]:
#MinHash algorithm
                hashed_ascii = (1*ascii_value + 2) % 3001

468

1784220410

  Created wheel for fnvhash: filename=fnvhash-0.1.0-py3-none-any.whl size=2173 sha256=502367a75f9f66cda4ffe10878dc06c2b3f105727010a84dceb0c4f99c91c823
  Stored in directory: /Users/josefholmgren/Library/Caches/pip/wheels/85/5c/08/9e08ae449df44376693bab5ef11f2a1254e865402348f3cfb0
Successfully built fnvhash
Note: you may need to restart the kernel to use updated packages.
